In [1]:
import mysql.connector as mysql
from config import host, user, password
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
con = mysql.connect(user=user,password=password,host=host,database='team3')

In [3]:
cursor = con.cursor()

## Co zrobić, żeby nie zbakrutować?

Początkowo chcemy na podstawie wydatków i zarobków <b>tylko</b> w kwietniu 2022, aby nasz bilans wyniósł 0.

Zacznijmy od sprawdzenia wydatków, których nie udało nam się pokryć w kwietniu 2022 z opłat za członkostwo.

In [4]:
query5 = "SELECT SUM(financial_flow) FROM finances WHERE date LIKE '2022-04-%'"
cursor.execute(query5)
results5 = cursor.fetchone()

In [5]:
debt = results5[0]

In [6]:
print("Nasz bilans w kwietniu 2022 wyniósł %.2f$." % debt)

Nasz bilans w kwietniu 2022 wyniósł -460665.30$.


### 1. Zakładając, że nasz jednyny zarobek to gracze, ilu nowych graczy potrzebujemy, aby pokryć te nadmiarowe wydatki z kwietnia 2022?

In [7]:
print("Potrzebowalibyśmy {} nowych graczy, którzy są tylko juniorami.".format(int(np.ceil(np.abs(debt/20)))))

Potrzebowalibyśmy 23034 nowych graczy, którzy są tylko juniorami.


In [8]:
print("Potrzebowalibyśmy {} nowych graczy, którzy są tylko dorośli.".format(int(np.ceil(np.abs(debt/55)))))

Potrzebowalibyśmy 8376 nowych graczy, którzy są tylko dorośli.


In [9]:
print("Potrzebowalibyśmy {} nowych graczy juniorów i {} nowych graczy dorosłych. (Przy założeniu, że każda grupa graczy pokryłaby taką samą część nadmiarowych wydatków.)".format(int(np.ceil(np.abs(debt/2/20))), int(np.ceil(np.abs(debt/2/55)))))

Potrzebowalibyśmy 11517 nowych graczy juniorów i 4188 nowych graczy dorosłych. (Przy założeniu, że każda grupa graczy pokryłaby taką samą część nadmiarowych wydatków.)


In [10]:
i = 1
while True:
    if 20*i + 55* i > np.abs(debt):
        break
    i += 1

In [11]:
print("Potrzebowalibyśmy {} nowych graczy juniorów i {} nowych graczy dorosłych. (Przy założeniu, że dołączyłoby tyle samo nowych zawodników z obu grup wiekowych.)".format(i, i))

Potrzebowalibyśmy 6143 nowych graczy juniorów i 6143 nowych graczy dorosłych. (Przy założeniu, że dołączyłoby tyle samo nowych zawodników z obu grup wiekowych.)


A ilu właściwie mamy graczy w tym momencie?

In [12]:
query6 = "SELECT COUNT(*) FROM personal_info WHERE position IS NULL"
cursor.execute(query6)
results6 = cursor.fetchone()
results6[0]

370

Niestety patrząc na to, że aktualnie mamy zaledwie 370 zawodników, ciężko byłoby nam zdobyć parę czy parędziesiąt tysięcy nowych graczy. Musimy więc znaleźć inne źródło zarobku. Zatem...

### 2. Jakiej wysokości musiałaby być nagroda za każdy wygrany mecz w kwietniu 2022, aby pokryć nadmiarowe wydatki z tego miesiąca?

Jak wiemy, za wygrane mecze czasami możemy otrzymać nagrody pieniężne, których dotąd nie roważałyśmy. Przyjmijmy więc, że otrzymaliśmy nagrodę w każdym meczu, który wygraliśmy w kwietniu 2022.

Ile meczy udało nam się wygrać w kwietniu 2022?

In [13]:
query7 = "SELECT COUNT(*) FROM matches WHERE (date LIKE '2022-04-%') AND (team_score > opponent_score)"
cursor.execute(query7)
results7 = cursor.fetchone()

In [14]:
won_matches = results7[0]

In [15]:
print("W kwietniu 2022 udało nam się wygrać {} meczy.".format(won_matches))

W kwietniu 2022 udało nam się wygrać 27 meczy.


In [16]:
print("Potrzebowalibyśmy za każdy wygrany mecz otrzymać {}$.".format(np.ceil(np.abs(debt/won_matches)*100)/100))

Potrzebowalibyśmy za każdy wygrany mecz otrzymać 17061.68$.


Oczywiście widzimy, że mamy raczej małe szanse, aby za każdą wygraną otrzymać kwotę 17 tysięcy dolarów. Pora zatem poszukać sponsorów.

### 3. Ilu potrzebowalibyśmy sponsorów na pokrycie nadmiarowych wydatków z kwietnia 2022?

Znalazłyśmy dane, które mówiły, że sponsoring drużyny sportowej to średnio od 100$ do 3000$. Rozważymy więc sponsorów, którzy każdej z naszych drużyn przekazują miesięcznie taką samą kwotę.

Załóżmy najpierw, że każdy ze sponsorów miesięcznie przekazywałby nam kwotę 100$ na drużynę.

In [17]:
print("Potrzebowalibyśmy {} takich sponsorów.".format(int(np.ceil(np.abs(debt/(100*43))))))

Potrzebowalibyśmy 108 takich sponsorów.


Ale jeśli każdy ze sponsorów miesięcznie przekazywałby nam kwotę 500$ na drużynę...

In [18]:
print("Potrzebowalibyśmy {} takich sponsorów.".format(int(np.ceil(np.abs(debt/(500*43))))))

Potrzebowalibyśmy 22 takich sponsorów.


A co gdybyśmy mieli bardziej hojnych sponsorów, którzy przekazywaliby miesięcznie 3000$ każdej drużynie?

In [19]:
print("Potrzebowalibyśmy już tylko {} takich sponsorów.".format(int(np.ceil(np.abs(debt/(3000*43))))))

Potrzebowalibyśmy już tylko 4 takich sponsorów.


Dość logiczne jest, że nowych zawodników pojawi się tylko ograniczona liczba, dlatego znacząco nie zwiększą oni naszych zysków. Ponadto, aby zająć się kolejnymi graczami, potrzebowalibyśmy więcej pracowników, co ponownie zwiększyłoby nasze wydatki.

Zdecydowanie najlepszym rozwiązaniem byłoby znalezienie sponsorów. Oczywiście ciężko pozyskać takich, którzy płaciliby kilka tysięcy dolarów miesięcznie każdej z naszych drużyn, jednak jacykolwiek sponsorzy pozytywnie wpłynęliby na bilans naszego konta.

Oczywiście powinniśmy też zacząć brać udział w meczach, w których za zwycięstwo moglibyśmy otrzymać nagrody pieniężne, a dodatkowo wygrywając - przekonać do sponsorowania nas.

## Co możemy powiedzieć o naszych pracownikach i zawodnikach?

### 1. Analiza ilości aktywnych pracowników na różnych stanowiskach w danych placówkach.

In [20]:
query8 = '''SELECT city, position, COUNT(*)
FROM people
LEFT JOIN gender
USING(first_name)
LEFT JOIN address
USING(person_id)
LEFT JOIN address_book
USING(address_id)
LEFT JOIN personal_info
USING(person_id)
WHERE position IS NOT NULL
AND retire_date IS NULL
GROUP BY city, position'''
cursor.execute(query8)
results8 = cursor.fetchall()

In [21]:
employees = pd.DataFrame(results8, columns=["city", "position", "amount"])

In [22]:
def make_position(df, col:int):
    fig.add_trace(go.Pie(values=df['amount'], labels=df['position']), 1, col)
    fig.update_traces(textinfo="value")

In [23]:
fig = make_subplots(rows=1, cols=5, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}]], subplot_titles=employees["city"].unique())
i = 1

for city in employees["city"].unique():
    make_position(employees[employees["city"]==city], i)
    i += 1

fig.update_layout(title_text="Podział wszystkich aktywnych pracowników w poszczególnych placówkach ze względu na stanowisko", title_x=0.5)
fig.show()

Widzimy, że największe grupy pracowników stanowią managerowie, sprzątaczki oraz trenerzy. Managerów mamy dokładnie tylu, ile drużyn w danej placówce. W każdej placówce mamy dokładnie jednego dyrektora. Ilość księgowych, psychologów oraz lekarzy w każdej z placówek waha się między 1 a 3.

### 2. Analiza ilości aktywnych graczy w placówkach ze względu na płeć.

In [24]:
query9 = '''SELECT city, gender, COUNT(*)
FROM people
LEFT JOIN gender
USING(first_name)
LEFT JOIN address
USING(person_id)
LEFT JOIN address_book
USING(address_id)
LEFT JOIN personal_info
USING(person_id)
WHERE position IS NULL
AND retire_date IS NULL
GROUP BY city, gender'''
cursor.execute(query9)
results9 = cursor.fetchall()

In [25]:
players_gender = pd.DataFrame(results9, columns=["city", "gender","amount"])
players_gender = players_gender.replace(['M'],'male')
players_gender = players_gender.replace(['F'],'female')

In [26]:
def make_gender(df, col:int):
    fig.add_trace(go.Pie(values=df['amount'], labels=df['gender']), 1, col)
    fig.update_traces(textinfo="value")

In [27]:
fig = make_subplots(rows=1, cols=5, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}]], subplot_titles=players_gender["city"].unique())
i = 1

for city in players_gender["city"].unique():
    make_gender(players_gender[players_gender["city"]==city], i)
    i += 1

fig.update_layout(title_text="Podział wszystkich aktywnych graczy w poszczególnych placówkach ze względu na płeć", title_x=0.5)
fig.show()

W Ottawie mamy najwięcej kobiet i stanowią one ponad 60% wszystkich aktywnych graczy w tej placówce. W pozostałych placówkach przeważają natomiast mężczyźni.

### 3. Analiza ilości graczy w placówkach, którzy są nadal aktywni bądź nie.

In [28]:
query10 = '''SELECT city, SUM(retire_date IS NULL), SUM(retire_date IS NOT NULL)
FROM people
LEFT JOIN personal_info
USING(person_id)
LEFT JOIN address
USING(person_id)
LEFT JOIN address_book
USING(address_id)
GROUP BY city
'''
cursor.execute(query10)
results10 = cursor.fetchall()

In [29]:
players_activity = pd.DataFrame(results10, columns=["city", "active", "retired"])
players_activity = players_activity.astype({"active": int, "retired":int})

In [30]:
def make_position(df, col:int):
    fig.add_trace(go.Pie(values=[df[['active']].iloc[0,0],df[['retired']].iloc[0,0]], labels=["active", "retired"]), 1, col)
    fig.update_traces(textinfo="value")

In [31]:
fig = make_subplots(rows=1, cols=5, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}]], subplot_titles=players_activity["city"].unique())
i = 1

for city in players_activity["city"].unique():
    make_position(players_activity[players_activity["city"]==city], i)
    i += 1

fig.update_layout(title_text="Podział wszystkich graczy w poszczególnych placówkach ze względu na bycie aktywnym", title_x=0.5)
fig.show()

Widzimy, że w każdej z placówek około 25% zawodników odeszło, a około 75% to nadal aktywni gracze. Ilościowo najwięcej graczy odeszło w Montrealu, natomiast procentowo - w Toronto. Wszędzie ilość zawodników, którzy odeszli waha się w przedziale od 25 do 35.